In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191


import matplotlib.pyplot as plt
import seaborn as sns

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

from scipy.optimize import minimize

# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
###加载数据

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:2704  pid:16344


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def nan_count(x):
    return round(x.isna().sum(axis=0)/len(x),4)*100

def filter_invalid_factor(fators_df):
    ## 缺失30%以上，且3倍方差以上，废弃
    ## 加极端判断是为了保证留下那种大窗口期的因子。
    
    fators = fators_df.replace([np.inf, -np.inf], np.nan)
    
    #计算缺失率
    valid_rate = excute_for_multidates(fators, nan_count, level=1) 
    valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*3
    #缺失率符合当前因子在所有标的上，符合3个标准差的
    filter_contition = valid_rate > valid_distribute_boundary 
    
    def filter_invalid(x):
        code = x.index.get_level_values(1)[0]
        distribute_condition = filter_contition.loc[code] 
        #符合3个标准差的列
        invalid_factors_name = distribute_condition[distribute_condition==True].index 
        #符合3个标准差，并且缺失30%以上的列
        thredhold_condition = valid_rate.loc[code][invalid_factors_name]>30 
        x[thredhold_condition.index] = np.nan 
        return x

    filtered = excute_for_multidates(fators, filter_invalid, level=1)
    
    return filtered

factors_filtered = filter_invalid_factor(res)

In [4]:
def filter_correlative_factor(fators_df, thredhold=.70, show_chart=False,verbose=False):
    #去掉相关性强的因子，相关性大于thredhold的因子（列）将被去除
    corr = fators_df.corr()
    matrix = corr.values
    #取得下三角矩阵
    matrix[np.triu_indices(matrix.shape[0])]=np.nan
    corr_triu = pd.DataFrame(matrix, columns=corr.columns, index=corr.index)

    if show_chart:
        plt.figure(figsize=(30,30))
        labels = corr.columns.str.replace('191_alpha','')
        ax = sns.heatmap(corr_triu, xticklabels=labels, yticklabels=labels, cmap="YlGnBu",cbar=True)

    # 取得符合条件的因子对后，再取“一对的左边”作为删除目标
    correlative_columns_name = corr_triu.abs().gt(thredhold).stack().loc[lambda x: x].index.get_level_values(0).unique().to_list()
    
    if verbose:
        print('delete correlative factors: {}'.format(','.join(correlative_columns_name)))
        
    correlative_droed = fators_df.drop(columns=correlative_columns_name)
    return correlative_droed

factors_filtered = filter_correlative_factor(factors_filtered,show_chart=False,verbose=True)

delete correlative factors: 191_alpha008,191_alpha014,191_alpha020,191_alpha022,191_alpha024,191_alpha026,191_alpha027,191_alpha028,191_alpha031,191_alpha034,191_alpha037,191_alpha040,191_alpha046,191_alpha047,191_alpha049,191_alpha050,191_alpha051,191_alpha052,191_alpha053,191_alpha055,191_alpha057,191_alpha058,191_alpha059,191_alpha062,191_alpha063,191_alpha065,191_alpha066,191_alpha067,191_alpha068,191_alpha071,191_alpha072,191_alpha078,191_alpha079,191_alpha082,191_alpha088,191_alpha089,191_alpha090,191_alpha094,191_alpha095,191_alpha096,191_alpha097,191_alpha099,191_alpha100,191_alpha105,191_alpha106,191_alpha112,191_alpha116,191_alpha121,191_alpha122,191_alpha126,191_alpha128,191_alpha129,191_alpha132,191_alpha133,191_alpha134,191_alpha136,191_alpha137,191_alpha139,191_alpha143,191_alpha147,191_alpha150,191_alpha153,191_alpha154,191_alpha156,191_alpha160,191_alpha161,191_alpha162,191_alpha165,191_alpha167,191_alpha168,191_alpha173,191_alpha174,191_alpha175,191_alpha177,191_alpha1

In [5]:
##标准化
## factors_filtered = (factors_filtered-factors_filtered.mean(axis=0,skipna=True))/  factors_filtered.std(axis=0,skipna=True)
fenlei = factors_filtered.nunique()
rows_count = factors_filtered.shape[0]
if rows_count > 1000:
    thredhold = rows_count *0.01
else:
    thredhold = 10
is_label = fenlei<thredhold
is_not_label = factors_filtered.columns.difference(is_label[is_label].index,sort=False)
is_not_label_factors = factors_filtered[is_not_label]
factors_filtered[is_not_label] = (is_not_label_factors-is_not_label_factors.mean(axis=0,skipna=True))/  is_not_label_factors.std(axis=0,skipna=True)


# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(factors_filtered.loc['2019-12-23'])
# factors_filtered['191_alpha075'].sort_values().unique()

In [6]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):

# factors = factors_filtered.iloc[:,0:112]# .loc[['2019-12-25']]
factors = factors_filtered

codes = factors.index.get_level_values(1).unique()
dates = factors.index.get_level_values(0).unique()
data_replenish = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
# data_replenish = smpl.get_data(codes.to_list(),'2019-12-25','2019-12-25').data
# ret = smpl.get_forward_return(data_replenish,'close')
ret = smpl.get_current_return(data_replenish,'close')
ret.name = 'ret'


mv_industry = smpl.add_marketvalue_industry(data_replenish).drop(columns=data_replenish.columns.to_list())
industy = mv_industry['industry']
cap = mv_industry['totalCapital']


# factors = factors.assign(cap=cap,industry = industy)
factors = factors.assign(country=1.0, cap=cap,industry = industy)
factors = pd.get_dummies(factors,'industry')

## 国家因子换到第一列
columns_ = factors.columns.to_list()
columns_.insert(0,columns_.pop(columns_.index('country')))
factors = factors.reindex(columns=columns_)



# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# del factors['country']


In [13]:


def multi_factors_model(factors_df,ret_series, model_confirm=False):
    ##factors_df 为单日，或多日混合，无date index。
    _invalid_check = nan_count(factors_df)
    ## 去除缺失5%以上的因子
    _invalid_checked = factors_df.drop(columns=_invalid_check[_invalid_check>5].index)
    ## 去除仍有少量缺失的标的
    _invalid_checked = _invalid_checked.dropna()
    ## 去除内容全部相同的列    
    _invalid_checked = _invalid_checked.loc[:, (_invalid_checked != _invalid_checked.iloc[0]).any()]
    ## 如果有国家因子需要加回来
    if 'country' in factors_df.columns:
        _invalid_checked = _invalid_checked.assign(country=1.0)
        ## 国家因子换到第一列
        _columns = _invalid_checked.columns.to_list()
        _columns.insert(0,_columns.pop(_columns.index('country')))
        _invalid_checked = _invalid_checked.reindex(columns=_columns)
        
    ## 去除内容列后，从新计算剩余的行业数量
    industy_count = _invalid_checked.columns.str.contains('industry_').sum()
    
    

    _cap=_invalid_checked['cap']

    #计算个股权重矩阵
    ##个股权重矩阵，权重为流通市值平方占比
    cap_temp = np.sqrt(_cap)
    wight = cap_temp/cap_temp.sum()
    V = np.diag(wight) #pd.DataFrame(np.diag(wight),columns = _cap.index,index = _cap.index)

    _industy_dummy = _invalid_checked.iloc[:,-1*industy_count:]
    industry_cap = _cap.T.dot(_industy_dummy)
    industry_wight = industry_cap/industry_cap.sum()    
    #=========================基础处理结束
    
    X = _invalid_checked[_invalid_checked.columns.difference(['cap'],sort=False)]

    ##计算约束矩阵防止共线性问题
    #构建约束矩阵R，R为K*K-1矩阵，K为因子数量
    #将最后一个行业因子表示为其他行业的线性形式
    #移除最后一个行业所在的列，并对其所在的行进行权重处理
    k = X.shape[1] #总因子数量
    p = industy_count #行业因子数量
    # q = k-1-p #风格因子数量
    q = k-p #原风格因子数量

    diag_R = np.diag(np.ones(k))
    location = q
    R = np.delete(diag_R,location+p-1,axis = 1)
    adj_industry_weights = -industry_wight.div(industry_wight[-1]).iloc[:-1]
    R[location+p-1,location:location+p] = adj_industry_weights


    # showR = pd.DataFrame(R,columns=np.delete(X.columns,location+p-1),index=X.columns)
    # with pd.option_context("display.max_rows", None, "display.max_columns", None):
    #     display(showR)
    # showR.to_excel('./showR.xlsx')

    #求解投资组合股票权重矩阵
    Y = ret_series[X.index]
    W = R.dot(np.linalg.inv(np.array(R.T.dot(X.T).dot(V).dot(X).dot(R),dtype='float'))).dot(R.T).dot(X.T).dot(V)
    W = pd.DataFrame(W,index = X.columns,columns = X.index)#.round(6)
 
    #计算因子收益率
    f = W.dot(Y).T
    if model_confirm:
        with pd.option_context("display.max_rows", None, "display.max_columns", None):
            display(W.dot(X).round(4))
    residual = Y - X.dot(f.T)
    residual.name='residual'
    # residual.sum()
    return residual

d='2019-11-20'
#  '191_alpha005',181 182 184 186 188 190 191 
# del factors['191_alpha005']
removed = factors.columns.difference(['191_alpha005','191_alpha006','191_alpha181','191_alpha184','191_alpha186','191_alpha188','191_alpha190','191_alpha191'],sort=False)

# del factors['191_alpha181']

# del factors['191_alpha182']
# del factors['191_alpha184']
# del factors['191_alpha186']

# del factors['191_alpha188']
# del factors['191_alpha190']
# del factors['191_alpha191']
re = multi_factors_model(factors.loc[d][removed],ret.loc[d], model_confirm=True)
re
# multi_factors_model(factors,ret)


,country,191_alpha001,191_alpha002,191_alpha003,191_alpha004,191_alpha007,191_alpha009,191_alpha010,191_alpha011,191_alpha012,191_alpha013,191_alpha015,191_alpha016,191_alpha017,191_alpha018,191_alpha019,191_alpha021,191_alpha023,191_alpha025,191_alpha029,191_alpha030,191_alpha032,191_alpha035,191_alpha036,191_alpha038,191_alpha039,191_alpha041,191_alpha042,191_alpha043,191_alpha044,191_alpha045,191_alpha048,191_alpha054,191_alpha056,191_alpha060,191_alpha061,191_alpha064,191_alpha069,191_alpha070,191_alpha073,191_alpha074,191_alpha075,191_alpha076,191_alpha077,191_alpha080,191_alpha081,191_alpha083,191_alpha084,191_alpha085,191_alpha086,191_alpha091,191_alpha092,191_alpha093,191_alpha098,191_alpha101,191_alpha102,191_alpha103,191_alpha104,191_alpha107,191_alpha108,191_alpha109,191_alpha110,191_alpha111,191_alpha113,191_alpha114,191_alpha115,191_alpha117,191_alpha118,191_alpha119,191_alpha120,191_alpha123,191_alpha124,191_alpha125,191_alpha127,191_alpha130,191_alpha131,191_alpha135,191_alpha141,191_alpha142,191_alpha144,191_alpha145,191_alpha146,191_alpha148,191_alpha151,191_alpha152,191_alpha155,191_alpha157,191_alpha158,191_alpha159,191_alpha163,191_alpha164,191_alpha166,191_alpha169,191_alpha170,191_alpha171,191_alpha172,191_alpha176,191_alpha178,191_alpha179,191_alpha182,industry_PCB,industry_中药,industry_乳制品,industry_云软件服务,industry_产联网信息服务,industry_住宅开发,industry_保险,industry_光伏电池组件,industry_光伏硅料,industry_光伏硅片,industry_光伏辅材,industry_其他IT设备,industry_其他专业工程,industry_其他广告营销,industry_其他建材,industry_军工电子,industry_冰洗,industry_功率半导体,industry_动力煤,industry_化学制剂,industry_化学工程,industry_医疗研发外包,industry_医院,industry_半导体封测,industry_半导体设备,industry_原料药,industry_厨房小家电,industry_品牌消费电子,industry_商用载货车,industry_啤酒,industry_国有大型银行,industry_城商行,industry_基因药物,industry_基建工程,industry_安防设备,industry_家具,industry_工程机械整机,industry_工程机械零件,industry_底盘与发动机系统,industry_快递,industry_房屋建设,industry_文化用品,industry_新兴软件,industry_旅游服务,industry_有机化工原料,industry_有机硅,industry_机场,industry_板材,industry_核力发电,industry_氨纶,industry_氯碱,industry_水产饲料,industry_水力发电,industry_水泥制造,industry_汽车电子电气系统,industry_油气开采,industry_油田服务,industry_消费电子组件,industry_涤纶,industry_清洁小家电,industry_港口,industry_游戏,industry_激光设备,industry_火力发电,industry_炼油化工,industry_焦炭加工,industry_煤化工,industry_燃气,industry_特钢,industry_玻纤制造,industry_生猪养殖,industry_电信运营服务,industry_电动乘用车,industry_电池化学品,industry_电网自动化,industry_畜禽饲料,industry_白酒,industry_稀土,industry_空调,industry_系统设备,industry_综合乘用车,industry_聚氨酯,industry_肉制品,industry_股份制银行,industry_航海装备,industry_航空装备,industry_航空运输,industry_航运,industry_血液制品,industry_行业应用软件,industry_证券,industry_诊断服务,industry_调味品,industry_车身附件及饰件,industry_轨交设备,industry_轮胎轮毂,industry_输变电设备,industry_配电设备,industry_钛白粉,industry_钴,industry_铁路运输,industry_铜,industry_铝,industry_锂,industry_锂电池,industry_防水材料,industry_集成电路设计,industry_面板,industry_风电整机,industry_黄金
country,0.9999,0.0000,-0.0001,0.0000,0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,-0.0010,-0.0000,0.0,0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0075,0.0091,0.006,0.0038,0.0048,0.0266,0.1011,0.0001,0.0016,0.0038,0.0007,0.0054,0.0018,0.0029,0.0012,0.0023,0.0038,0.0035,0.0149,0.0178,0.001,0.0065,0.0012,0.0011,0.0012,0.0016,0.0021,0.0019,0.0005,0.0032,0.1412,0.0103,0.0028,0.0156,0.0122,0.0014,0.0068,0.0013,0.0034,0.0094,0.0096,0.0015,0.0024,0.0056,0.0005,0.0009,0.0049,0.0040,0.0025,0.0003,0.0008,0.0017,0.0189,0.0082,0.0013,0.0335,0.0026,0.0201,0.003,0.0004,0.0043,0.0034,0.0012,

date        code  
2019-11-20  000001    2.286705e-04
            000002    2.339543e-04
            000063    5.103867e-08
            000069   -7.908760e-04
            000100   -7.058520e-05
                          ...     
            603806   -3.930844e-08
            603833   -7.996397e-09
            603882   -4.642294e-07
            603899    6.293644e-07
            603986    1.211932e-04
Name: residual, Length: 212, dtype: float64

In [8]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(X.isna().sum(axis=0)>0)
# test
# test_invalid_check = nan_count(test)
# test_invalid_checked = test.drop(columns=test_invalid_check[test_invalid_check>5].index)
# test_invalid_checked.dropna()

# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     # display(pd.DataFrame(R.T.dot(X.T),columns=X.index,index=showR.columns).sum(axis=1))
#     display(pd.DataFrame(R.T.dot(X.T).dot(V).dot(X).dot(R),columns=showR.columns,index=showR.columns))
    
    # display(X.sum(axis=0))
    # display(R.T.dot(X.T))
# W

# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(factors['191_alpha005'].loc[d])
# (res['191_alpha063']-res['191_alpha063'].mean())/res['191_alpha063'].std()

# factors.columns.to_list()

In [ ]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(f)

def factor_test(date_):
    ## 外部 factors，ret
    test_factor = factors.loc[date_]
    test_ret = ret.loc[date_]

    residual = multi_factors_model(test_factor,test_ret)

    retcurrent_index = retindex_all.index(pd.Timestamp(date_))
    retdate_range = retindex_all[retcurrent_index+1:retcurrent_index+4]
    
    
    return pd.DataFrame(residual)
    # residual_df = pd.DataFrame(residual.abs())
    # try:
    #     residual_df['ret3'] = residual_df.apply(lambda x:ret.loc[(retdate_range,x.name)].sum(),axis=1)
    # except Exception as e:
    #     print(date_,e.__class__.__name__,e)
    #     # traceback.print_exc()
    #     return None
    # return residual_df.sort_values(by='residual',axis='index', ascending=False)['ret3']
    


# date_start = '2019-01-07'
# step = 1
# retindex_all = ret.index.get_level_values(0).unique().to_list()[0::step]
# factors_date_all = factors.index.get_level_values(0).unique().to_list()[0::step]

# test_date_index = factors_date_all.index(pd.Timestamp(date_start))
# test_date_range = factors_date_all[test_date_index+1:test_date_index+int(710/step-1)] #710
# # test_date_range


# # res_cum_pos = pd.Series(0,index=factors.index.get_level_values(1).unique())
# # res_cum_neg = pd.Series(0,index=factors.index.get_level_values(1).unique())
# res_cum = []
# for date_ in test_date_range:
#     res = factor_test(date_)
#     if not res is None:
#     #     res_ = res.dropna()
#     #     topN = res_[0:1]
#     #     tailN = res_[-1:]
#     #     res_cum_pos[topN.index] = res_cum_pos[topN.index]+topN
#     #     res_cum_neg[tailN.index] = res_cum_neg[tailN.index]+tailN
#         res_ = res.dropna()
#         res_cum.append(res_.mean())
#         if res_.mean().abs()[0] > 1:
#             print(date_,res_.mean())
# # print(np.sum(pos),np.sum(neg))


In [78]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
# #     # display(X.dot(f.T))
# #     # display(residual.sort_values(ascending=1))
#     display(res_cum)

# res_cum_pos.mean(),res_cum_neg.mean()

a=np.array(res_cum).reshape(-1)
a.sort()
print(a.mean(), a.std()) #0.007843376600751448 0.6770983120962308

0.005935451783332656 0.2086955443641455


In [15]:
# bench=smpl.get_index_data('399300',start=str(test_date_range[0])[0:10],end=str(test_date_range[-1])[0:10])
# bench_close = bench.data.close
# (bench_close[-1]-bench_close[0])/bench_close[0]
# bench.data


# factors.loc[['2019-01-07','2019-01-08']].reset_index('date',drop=True).sort_index()

# tt_f = factors.loc[b1]
# tt_r = ret.loc[b1]

# # with pd.option_context("display.max_rows", None, "display.max_columns", None):399  195 197
#     # display(multi_factors_model(tt_f,tt_r))

# re = multi_factors_model(tt_f,tt_r)
re.loc[(slice(None),'000333')]

date
2019-12-25    0.007705
Name: residual, dtype: float64

In [63]:
# 特定风格因子的预期暴露度最大化

def style_factors_weight_model(factors_df,ret_series):
    ##factors_df 为单日，或多日混合，无date index。
    _invalid_check = nan_count(factors_df)
    ## 去除缺失5%以上的因子
    _invalid_checked = factors_df.drop(columns=_invalid_check[_invalid_check>5].index)
#     ## 去除仍有少量缺失的标的
    _invalid_checked = _invalid_checked.dropna()
    ## 去除内容全部相同的列    
    _invalid_checked = _invalid_checked.loc[:, (_invalid_checked != _invalid_checked.iloc[0]).any()]
    ## 如果有国家因子需要加回来
    if 'country' in factors_df.columns:
        _invalid_checked = _invalid_checked.assign(country=1.0)
        ## 国家因子换到第一列
        _columns = _invalid_checked.columns.to_list()
        _columns.insert(0,_columns.pop(_columns.index('country')))
        _invalid_checked = _invalid_checked.reindex(columns=_columns)
        
#     ## 去除内容列后，从新计算剩余的行业数量
    industy_count = _invalid_checked.columns.str.contains('industry_').sum()

    _cap=_invalid_checked['cap']

#     #计算个股权重矩阵
#     ##个股权重矩阵，权重为流通市值平方占比
    cap_temp = np.sqrt(_cap)
    wight = cap_temp/cap_temp.sum()
    # V = np.diag(wight) #pd.DataFrame(np.diag(wight),columns = _cap.index,index = _cap.index)

    _industy_dummy = _invalid_checked.iloc[:,-1*industy_count:]
    industry_cap = _cap.T.dot(_industy_dummy)
    industry_wight = industry_cap/industry_cap.sum()
    
    #=========================基础处理结束

    def w_style_func(w, X, style_loc=1):
        '''
        w: 求解的最优权重[w1, w2, ..., wn]^T
        X: 投资组合个股风格因子暴露度矩阵
        H: 投资组合个股行业因子哑变量矩阵
        '''
        return -np.dot(w, X[:, style_loc])


    
    # print(_invalid_checked.columns)
    X = _invalid_checked[_invalid_checked.columns.difference(['cap'],sort=False)]
    X = X.iloc[:,1:-1*industy_count] # 表示N只股票的风格因子暴露度矩阵
    
    # with pd.option_context("display.max_rows", None, "display.max_columns", None):
    #     display(X)
    # print(X)
    
    Xs = X.values
    H = _industy_dummy.values # 表示N只股票的行业因子哑变量矩阵
    h = industry_wight.values # 表示基准指数对应的行业权重，与所选取的参照基准有关
    
    print(h)
    print(X.shape)
    print(np.average(ret_series[X.index]))
    
    num = X.shape[0]                   # 股票数目
    w_ = np.random.random_sample(num)    # 生成介于[0, 1]之间的随机数
    init_w = w_ / w_.sum()               # 随机初始化权重值
    bnds = ((0, 1), ) * num       # 权重限定位于[0, 1]之间
    res_lst = []
    for loc in range(Xs.shape[1]):
        print('-->', loc, X.columns[loc])
        X_ = np.delete(Xs, loc, axis=1)
        cons = (
            # {'type': 'eq', 'fun': lambda w: np.dot(w, X_)},            # 数值为0的向量，向量长度等于风格因子数目减1
            {'type': 'eq', 'fun': lambda w: np.dot(w, Xs[:, loc]) - 1}, # 对本身的风格暴露度限定为1
            {'type': 'eq', 'fun': lambda w: np.dot(w, H) - h},          # 对行业因子暴露度与基准一致，由于不能做空，所以无法确保暴露度为0
            {'type': 'eq', 'fun': lambda w: sum(w)-1},                 # 不支持卖空，所以权重之和等于1，其中权重大于0的约束放在参数bounds里面
            )  

        res = minimize(w_style_func, x0=init_w, args=(Xs, loc), constraints=cons, bounds=bnds, method='SLSQP', options={'disp': True,'maxiter':50})
        # res = minimize(w_style_func, x0=init_w, args=(Xs, loc), constraints=cons, method='COBYLA', options={'disp': True,'maxiter':100})
        print(np.dot(res.x,ret_series[X.index]))
        res_lst.append(res.x)
        if loc==5:break
    return np.vstack(res_lst)

#["191_alpha001","191_alpha007","191_alpha009","191_alpha010","191_alpha012","191_alpha018","cap"]
d='2019-12-05'
# removed = factors.columns.difference(['191_alpha103','191_alpha104','191_alpha107','191_alpha004','191_alpha006','191_alpha075','191_alpha016','191_alpha044','191_alpha164','191_alpha171','191_alpha181','191_alpha182','191_alpha184','191_alpha186','191_alpha188','191_alpha190','191_alpha191'],sort=False)
removed=factors.columns.difference(factors.columns[4:105])
removed = removed.difference(['191_alpha182'],sort=False)

rex = style_factors_weight_model(factors.loc[d][removed],ret.loc[d])
rex


[0.0076983  0.0089964  0.00635458 0.00382021 0.00494083 0.02856265
 0.11189134 0.00053027 0.00183848 0.00438843 0.00077338 0.00551237
 0.00192258 0.00130228 0.00243411 0.00374554 0.00428064 0.0183947
 0.01673504 0.0010565  0.00618874 0.00114563 0.00123606 0.00139254
 0.00161174 0.00207435 0.00193562 0.00060395 0.0032375  0.0849328
 0.01921208 0.00261015 0.01670814 0.01181929 0.00146212 0.0014231
 0.00733416 0.00135301 0.00381704 0.00932398 0.01011771 0.00154647
 0.00245519 0.00580982 0.00055609 0.00092164 0.00509392 0.00414828
 0.00261431 0.00035276 0.00087594 0.00172777 0.01981283 0.00892896
 0.00132794 0.03553665 0.00272351 0.02174708 0.00329899 0.00037436
 0.00450756 0.00369821 0.00135869 0.00316716 0.02407857 0.00095875
 0.00043161 0.00273408 0.00112453 0.00674017 0.00597221 0.00414364
 0.00298282 0.00329017 0.0030767  0.08130048 0.00127552 0.02620797
 0.00465769 0.01766626 0.00539901 0.00332665 0.09859833 0.00096103
 0.00461043 0.01120603 0.00202524 0.00230832 0.00340012 0.0608650

array([[6.46148558e-13, 2.54447649e-18, 4.65769288e-03, ...,
        1.54646683e-03, 7.27573088e-03, 2.69658613e-03],
       [0.00000000e+00, 2.10458606e-11, 6.18880694e-03, ...,
        1.11811564e-03, 0.00000000e+00, 4.48176885e-03],
       [1.06959944e-11, 3.35519842e-02, 6.32997651e-03, ...,
        1.07862142e-03, 2.54189551e-12, 4.64636369e-03],
       [0.00000000e+00, 3.05913276e-02, 5.33764786e-03, ...,
        1.35623964e-03, 1.37094763e-13, 3.48937088e-03],
       [2.26135984e-03, 6.29508050e-03, 7.52670712e-03, ...,
        7.43818819e-04, 2.68127083e-03, 6.04167632e-03],
       [1.16356456e-11, 0.00000000e+00, 5.86302785e-03, ...,
        1.20925703e-03, 0.00000000e+00, 4.10193091e-03]])

In [53]:
# re.index.get_level_values(1).unique()
factors.columns

Index(['country', '191_alpha001', '191_alpha002', '191_alpha003',
       '191_alpha004', '191_alpha005', '191_alpha006', '191_alpha007',
       '191_alpha009', '191_alpha010',
       ...
       'industry_铜', 'industry_铝', 'industry_锂', 'industry_锂电池',
       'industry_防水材料', 'industry_集成电路设计', 'industry_面板', 'industry_风力发电',
       'industry_风电整机', 'industry_黄金'],
      dtype='object', length=232)